### Welcome to zachFeeds - the Ultimate RSS Parser!

- zachFeeds takes in any given RSS Feed, and stores it into FireBase, allowing you to star your favorites, or just compile articles from hundreds of different feeds

- zachFeeds is written in Flask, a lightweight Python webserver, with Jinja2 and HTML as the frontend markup. The reason I did not use React or Angular is that both of those require running some version of Node.js on the host machine, and this is designed to be completely platform-agnostic, without the need for Docker!

- Flask was better than Django for this project because I did not need the heavy extra features of Django, and no complex routing or host-machine SQL is needed!

### How to run the project:
1. Import the modules from `requirments.txt`. These boil down to an XML reader, Flask, and Firebase access. 
2. Run ` python FlaskMain.py`
3. That will start the web server, which can be accessed in any modern web browser by navigating to `localhost:5000`
4. Tests should run autmatically, but if not, uncomment line 49 of `FlaskMain.py`, which contains `tests.runTests()`.

### Why Firebase, and why is my access key public!?

- I used Firebase because it is free, and has a super nice viewer. 
- My access key (in `firebaseKey.json`) is public because I needed the project to be runnable from any computer, and that was the easiest way. If and when this project enters production, I will make a new Firebase database and keep my keep hidden on the webserver. 

### How the backend works

- In order to get data, I have one continuous table in Firebase of RSS XML (XML is a markup language) links, that get iterated through every time the 'Feed' is updated!

- Each feed is located in my Firebase database uner `root/feeds`. 

- In order to get the data from these links, I use a Python module called requests, which will return the XML of every feed. I then use a library called XMLToDict to get all the objects from the response into a series of dictionaries within a dictionary, which is Python readable. then, I run the dictionary through a function to pull the Feed name, and then each entry, and save that into a new dict that just contains the title, and each entry as an array of objects, using `XMLManipulate.RawDataToDict`.

- Once the raw data has been parsed into something readable, and stripped of all the unnecessary data (unnecessary data is indicators such as weather it is Atom2 or standard RSS XML being returned from the site, my software does not care about this), I can then connect to my Firebase instance and push articles to it. 

- Connecting to Firebase is done through `UseFirebase.postArticles()`. This function takes in my formatted and parsed dict of articles and their data, and sends each one to Firebase as an individual opject, under `root/articles` in my storage container. However, there are two major slowdowns with this process, that could be improved: I should use some form of RegEx to parse the publish date when I am creating the article in the entries array in the dict, which will alleviate load on the server when it is pushing to Firebase, instead of making it a date, and then modifying that date with `streftime()`, and then pushing it. I also should change the initial article construction to just use the sub fields of each element, instead of having to drop down to them when pushing to Firebase, that way I am sending far less data around to different functions, and decreasing general server load when an update is run. 

- Once it is parsed into a dict, I upload the data from each article into my Firebase DB, and refresh the frontend.


### How the frontend works

- The frontend is written in Jinja2, an in-markup scripting language. Jinja allows me to pass Python variables as 'props', or in other words, construction data, to the page, and then access them through Jinja syntax. 

- Jinja syntax is fairly straightforward, with curly braces being used for any logic or variable-usage statements. `{{ }}` is used to get a variable, and `{% %}` is used for logic such as loops. the rest of the syntax is very Pythonic, save for the identifiers that can be used to render variables, To add a modifier, | is used, then the name of the mod. For example, if I wanted to render the article title string as HTML, I would write `{{ article.title|safe }}`. Moreover, for the article tityle specifically, they can get very long, and I do not need to render a wholem paragraph of text for each article, so I need to use the `truncate()` mod. To do this, it is written as `{{ article.title|safe|truncate(50) }}`. The 50 here represents the number of characters rendered, before it turns into an ellipses. 

- Each page is represented by a route in `FlaskMain.py`. 

### A run-through of what happens when the webapp gets run:

1. Upon running `python FlaskMain.py`, first Python will check over all the syntax, to make sure it checks out, as well as imports. Next, I define the app object to run and store all of the webapp's features. This is a new `Flask` object, which I pass `__name__` to, which is how Python can get data about how it is being run. Defining it this way allows me to lazyload, which makes Flask do more of the work than me having to handwrite more logic that is unnecessary for an app this small. Then, it checks over and defines every route, by first checking for any `@app.route()` decorators, and assigning the function below it to that route. 

2. Once the routes have been defined, the app runs `UseFirebase.init()`. This function establishes a connection to my Firebase database, in order to manipulate it. If the connection already exists, then it just prints it already is connected, and continues on. 

3. Once the DB is connected, I start a local webserver by running `app.run()`, which allows the server to be connected to locally through the web-browser on the port 5000, at either `127.0.0.1:5000`, or `localhost:5000`. These both mean the same thing. 

4. Once the server is started up, there are 3 viewable routes for the user: '/', which is the index, or homepage, and is defined as the 'root URL', '/feeds', which is where there is a list of every feed being used in the DB, and '/addfeed', which is where the user can use the form to add a new feed to the DB. 

##### For an explanation of how routes work, see below.

### How a route works: An In-Depth Explanation. 

For this examination I will use the root, or '/' route, as it is the most commonly used one on this site. 
Once the route is defined in Flask on start, whenever a user goes there in the browser, the following happens:

1. I set the variable `articles` to be a list of each article in the database, by calling a function from `UseFirebase`. 

2. Next, I sort the articles by their publish date, in order to make them more readable for the user. However, this results in ascending order, so I reverse it so newer articles are first. 

3. Next, I format the date in each article to an American-style readable date. I do this here rather than the backend so that it can be easily adapted to different webapps or formats without having to mass-change my database. 

4. Lastly, I run `render_template`, which renders a Jinja2 HTML template to the browser. Here, I tell it to render the `home.html` template, and pass it the articles as props data. 

5. After this, the route's job is done, and the logic then turns to Jinja. The `home` template extends from `base`, meaning that it is essentially an instance of it, with it's whole render taking place in `{% block content %}`. Doing it this way allows me to have one neavbar, and general CSS imports and Javascript, without rewriting it for every page. 

6. Then, the page runs through the `for loop`, which just iterates over, and formats articles into a table. 

The 3 pages work jusat like that, with variations depending on what they do. However, there is another type of route, which can be seen in '/update'. This route forces the browser to POST data to it, so the browser cannot render this route, and this prevents CORS requests (cross-origin attacks that can infect my server or the user's computer with malicious code). Moreover, to post this route, a third language is required, JavaScript. JavaScript is special because it can run logic from the browser, and manipulate the DOM (HTML document) without refreshing the page. It can also be run on the server via Node.JS and express.JS (the equivalent of Python and Flask), but I did not do that here. 

When the user clicks the 'Update' button on the home page, it runs the Javascript function `updateArticles()`, which can be found in the `<script>` tags at the bottom of the page. The function does the following:

1. First, it sets the actual content of the page to hidden by setting its `display` style to `none`. Next, I set the loading bar to display by chaning its `display` style to `block` (the CSS default). I get these elements by access the `document`, then the function `getElementsByClassName`, anmd the first element of each. I do this to show the user something happened when they clicked the button, and prevents them from spamming it 100 times. 

2. Next, I define a dictionary called opts via `let opts = {...}`. Unlike Python, JavaScript has two main declarators for creating variables: `let`, and `const`. These set the scopes and accessibility of the variables. There is a third, called `var`, but this is antiquated because variables created with this have no scope, and cannot be removed, even when the page changes, or a different function is run. The `opts` dict contains a method, set to POST, and and a headers dict, which is empty, but could be used to send data, like a login token, back to the server. 

3. Then, I run `fetch()`, which hits a route on a webserver, in this case, '/update', and I pass it the `opts` dict to tell it that it should POST the data. Afterwards, I call `.then`, and create a sub-function that returns a JSON version of the response string. `.then` is special because it enables me to use Promises, which are very special. Normally, JavaScript runs synchronously, or all at once, so Promises allow the code to stop and wait for a response, without needing a bunch of `async` functions. Regardless, this promise creates a function that returns the data from the backend Flask server. The next `.then()` function takes this response, and checks if it reads that the articles actually updated. If they did, it reloads the page, which will show the new ones. Else, it logs the error response to the console window, which is helpful for debugging. It knows this because the `update` function in Flask either returns "done", or the error it encountered while running. 

The other route that works like that is '/postfeed', which does essentially the same thing, but takes data from a user-entered form, and posts that to Firebase. 

### Why a Flask app over Django, or even a webapp at all?

- To start the question of why I made this a webapp, it is two pronged: First, I intend on hosting this on a small Raspberry Pi server or on Google Cloud host so that I can access it from anywhere. I could just make it an app for my phone, but to do that requires me to write a fully scalable UI in a library like Tkinter, and then port it to mobile using PyGame. moreover, that likely will not run well, as Python is not compiled, and I still could not access it from the desktop. Therefore, a webapp is just easier - HTML can render on literally any device with a web browser, or some without, for that matter, and because the logic is done server-side, that is all the device needs, and styling using HTML and CSS is fairly simple once you do it once, because it is entirely markup, with no real logic. Moreover, I am an experienced web developer (I have been making .NET, Node.JS, and Ruby on Rails apps for a very long time), and getting the UI up and running wouild not take a very log time, which I needed for this project. 

- As for the question of Flask VS. Django, that is a little bit more complex: 
    - At first glance, Django is the obvious choice, as it is an MVC (Model-View-Controller) platform. This means that, just like old-school ASP .NET and Ruby, the webapp is written in a very specific way: First is the model, which is how data is handled. I would create an article and feed class, each of which contains functions to send data to the (usually SQL) server, and can hold data while in use. Then, I would create a view, or route, which can contain still-backend logic to control the user, and then points to a controller, which is where data is finally passed to the page. Moreover, Django also comes prebaked with an admin page, loads of support, and an extraordinarily nice to use routing system and definition system, as well as its own organizational system. However, these features ended up being Django's achilles heel for this project... It is huge! The average Django app is about quadruple the size of my little Flask app, and that is without my RSS logic! It was simply not designed for these kinds of small projects, and to boot, the modern web is moving away from MVC structures, and towards route-view systems, with backend logic, similar to the way Node does it. And, on another note, it is kind of slow and bloated to work with, even in production envs...
    - So Django didn't work, but why did Flask? Well, to start, it is literally only one file! All of my web logic is contained in `FlaskMain.py`, and is super simple! I just defined a few routes, gave them some basic templates, and could call the frontend a day, which enabled me to focus on getting my logic just right. 

### So the logic and basic Javascript to control the website works now... but it needs styling!

- In order to style a page, designers usually do one of two things: Use a framework such as [BootStrap](https://getbootstrap.com/docs/4.0/getting-started/introduction/), or spin your own framework and theme for the site. Moreover, I am not very good at UI design, so I opted to use a framework, called [Materialize CSS](https://materializecss.com/about.html). This is Material Design (which I am partial to), and looks somewhat unique compared to the hundreds of thousands of Bootstrap pages out there. Moreover, it still has a very powerful flexbox Grid system for making the page responsive, and some nice pre-chosen colors for me to theme the page to my liking. I access it through a CDN (Content Delivery Network) in `base.html`. 

- Now that I had a framework, I just implemented a navbar using the materialize example, and gave it a sidenav by initializing it in Javascript in `base.html`. After that, I just needed to add their classes to my HTML elements to make them pretty, and then I was able to make my actions float in the bottom right as nice, animated buttons, known as Action Buttons in Material Design. 

##### OK, but what is Material Design?
- Material Design is a standard implemented by Google across Android, and all of their websites, designed to unify the UI and make it easy to use, and look pretty at the same. While using this standard can make most sites look fairly generic, it does make it easy for people such as myself, who are not great at graphic design, to make decent looking sites. 

### Minimum Viable Product: 

- Use requests to parse XML into my dict, and post it to Firebase, then retrieve that data. This can be seen below, with the complex version being a part of the site.

### See below for how most of the program logic works! The rest of the app is just Flask boilerplate, but this is the important bit.

In [6]:
#MVP imports... RUN ME FIRST
import requests #Get/Post data to and from web servers
import xmltodict #Parse raw XML data from RSS Feeds
import firebase_admin
from firebase_admin import credentials, db

In [5]:
# Get data from RSS Feed (In this case Google Alerts feed for COVID)

def getRawData(dataLink): #Get raw data
    return requests.get(dataLink).text #Requests returns raw XML data

def convertRawData(dataLink): #Convert raw XML to Dict
    rawData = getRawData(dataLink) #Get raw XML Data
    rawDict =  xmltodict.parse(rawData, dict_constructor=dict) #Convert to OrderedDict
    
    convertedDict = rawDict["feed"] #Root element of RSS Feed
    
    return convertedDict

def rawDataToDict(rawData): #Convert all the raw data into something usable
    finalDict = {
        "title": rawData["title"],
        "entries": []
    }
    
    for article in rawData["entry"]: #Entry is an array in the dict
        articleDict = { #Get necessary elements from current entry
            "title": article["title"]["#text"], 
            "link": article["link"]["@href"],
            "content": article["content"]['#text']
        }
        
        finalDict["entries"].append(articleDict) #Append current entry to the data
        
    return finalDict
        
# TEST WITH COVID-19 GOOGLE ALERT FEED
data = convertRawData("https://www.google.com/alerts/feeds/00698824683610429194/1098921106065488311")

current_data = rawDataToDict(data)

print(current_data["entries"][0]["title"]) #Get the title of the first article

Allegheny County sets new daily record for <b>COVID</b>-19 cases


In [7]:
# Please do not run, this does not check for duplicate articles like the live version does! Just for testing! 

# The database will be clogged with bad entries if this gets run! Check out UseFirebase.postArticles for a version with the check!

# Put data from dict into FireBase
cred = credentials.Certificate('firebaseKey.json') #Service Account for DB Access

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://zachfeeds-86449.firebaseio.com/'
})

ref = db.reference('articles') #Put articles

for article in current_data["entries"]: #For each article
    new_article_ref = ref.push({ #Post the data
        'title': article['title'],
        'link': article['link'],
        'content': article['content'],
        # Format ISO 8061 date into Python readable, and format it to human readble, then push to DB
        'published': str(datetime.strptime(article['published'], "%Y-%m-%dT%H:%M:%SZ").date()) 
    })




In [57]:
#Get data out of firebase, test to see if articles can be pulled!

ref = db.reference('articles') #get articles data

articles = ref.get()

storedArticles = [] #Array of stored articles to later iterate over

for article in articles: #For article in FB return
    currentArticle = {} 
    for elem in articles[article]: #For each element of the article
        currentArticle[elem] = articles[article][elem] #Append data to the dict
    storedArticles.append(currentArticle) #Append the dict to the array
    
print(storedArticles[0]['title']) #Test first bit of data in array

8 months after Iowa&#39;s first <b>COVID</b>-19 death, toll at 2224


In [6]:
#Add feeds to firebase, please do not run, this will also clog up the DB with dupe feeds! Check UseFirebase.addFeed for a vsn with the check, or use the site!

ref = db.reference('feeds') #get feeds data

new_feed_ref = ref.push("https://www.google.com/alerts/feeds/00698824683610429194/3310492921969815257")